In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
txt = ''
for file in os.listdir():
    if file.endswith(".txt"):
        with open(file, 'r', encoding="cp1251") as f:
            text = f.read()
            txt += text

In [3]:
len(txt)

6723492

In [4]:
##Уникальные символы Достоевского
chars = sorted(list(set(txt)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#&'()*,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXZ[]_abcdefghijklmnopqrstuvwxyz «»ІАБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЪЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёїќ–—’“”„…№
160


In [5]:
##char level tokenizer
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[c] for c in l])

In [6]:
data = torch.tensor(encode(txt), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([6723492]) torch.int64
tensor([109,  87, 103, 104, 113,   2, 101,  92, 102,  89,  87, 116,   1,   1,
          1,   1,  90, 128, 117, 119, 117,   2, 132, 122, 133, 119, 117, 148,
          1,   1,   1,   1,  37,   1,   1,  99, 122,   2, 136, 135, 122, 133,
        132, 122, 119,  11,   2, 148,   2, 134, 122, 128,   2, 124, 117, 132,
        125, 134, 144, 119, 117, 135, 145,   2, 146, 135, 136,   2, 125, 134,
        135, 131, 133, 125, 147,   2, 129, 131, 125, 138,   2, 132, 122, 133,
        119, 144, 138,   2, 141, 117, 120, 131, 119,   2, 130, 117,   2, 123,
        125, 124, 130, 122, 130, 130, 131, 129,   2, 132, 131, 132, 133, 125,
        142, 122,  11,   2, 135, 131, 120, 121, 117,   2, 127, 117, 127,   2,
        129, 131, 120,   2, 118, 144,   2, 131, 118, 131, 126, 135, 125, 134,
        145,   2, 125,   2, 118, 122, 124,   2, 135, 131, 120, 131,  13,   2,
        100, 121, 130, 131,   2, 124, 130, 117, 147,   2, 130, 117, 119, 122,
        133, 130, 131,  25,   

In [7]:
ttratio = 0.9

In [8]:
n = int(ttratio * len(data))
train = data[:n]
val = data[n:]

In [9]:
torch.manual_seed(42)
batch_size = 64
block_size = 128

def batch(split):
    data = train if split=='train' else val
    idx = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i+1:i + block_size+1] for i in idx])
    return x, y

In [10]:
xb, yb = batch('train')

In [11]:
xb

tensor([[122,  11,   2,  ..., 131,  11,  83],
        [127, 117, 127,  ..., 127, 117, 129],
        [  2, 125,   2,  ...,   2, 118, 122],
        ...,
        [119,   2, 140,  ..., 144,   2, 132],
        [131,   2, 146,  ..., 128, 125,   2],
        [119, 117, 128,  ..., 133, 122,  13]])

In [12]:
yb

tensor([[ 11,   2, 130,  ...,  11,  83, 152],
        [117, 127,  12,  ..., 117, 129, 125],
        [125,   2, 135,  ..., 118, 122, 134],
        ...,
        [  2, 140, 122,  ...,   2, 132, 133],
        [  2, 146, 135,  ..., 125,   2, 136],
        [117, 128, 125,  ..., 122,  13,   2]])

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
##And now, self-attention


n_layer = 8
n_embd = 128
n_head = 32

dropout = 0.2
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C **-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLM2(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        b, t = idx.shape
        
        tok_emb = self.token_embedding_table(idx) ##(b, t, c)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x).to(device) ##(b, t, vocab_size)

        
        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape
            logits = logits.view(b*t, c).to(device)
            targets = targets.view(b*t).to(device)

            loss = F.cross_entropy(logits, targets).to(device)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx.to(device)

xb_cuda = xb.to(device)
yb_cuda = yb.to(device)

m2 = BigramLM2().to(device)
print(sum(p.numel() for p in m2.parameters())/1e6, "M parameters")
logits, loss = m2(xb_cuda, yb_cuda)
print(logits.shape)
print(loss)

1.640864 M parameters
torch.Size([8192, 160])
tensor(5.2414, device='cuda:0', grad_fn=<NllLossBackward0>)


In [14]:
optim = torch.optim.AdamW(m2.parameters(), lr=3e-3)
sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9)

batch_size = 128
for steps in range(5000):
    xb, yb = batch("train")
    xb_cuda = xb.to(device)
    yb_cuda = yb.to(device)
    logits, loss = m2(xb_cuda, yb_cuda)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

    if steps % 200 == 0:
        sched.step()

    if steps % 100 == 0:
        print(f"Step number: {steps}, loss: {loss.item()}, LR: {sched.get_last_lr()}")


Step number: 0, loss: 5.243777751922607, LR: [0.0027]
Step number: 100, loss: 2.6064109802246094, LR: [0.0027]
Step number: 200, loss: 2.534100294113159, LR: [0.0024300000000000003]
Step number: 300, loss: 2.238802194595337, LR: [0.0024300000000000003]
Step number: 400, loss: 2.0308139324188232, LR: [0.002187]
Step number: 500, loss: 1.8573051691055298, LR: [0.002187]
Step number: 600, loss: 1.8129334449768066, LR: [0.0019683]
Step number: 700, loss: 1.7411245107650757, LR: [0.0019683]
Step number: 800, loss: 1.6971685886383057, LR: [0.00177147]
Step number: 900, loss: 1.6656464338302612, LR: [0.00177147]
Step number: 1000, loss: 1.6415959596633911, LR: [0.0015943230000000001]
Step number: 1100, loss: 1.6181708574295044, LR: [0.0015943230000000001]
Step number: 1200, loss: 1.5791606903076172, LR: [0.0014348907]
Step number: 1300, loss: 1.5825529098510742, LR: [0.0014348907]
Step number: 1400, loss: 1.5791224241256714, LR: [0.00129140163]
Step number: 1500, loss: 1.5874698162078857, LR:

KeyboardInterrupt: 

In [27]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m2.generate(context,max_new_tokens=2000)[0].tolist()))

	Это есть! – пробовал вдруг Нину, которого остащив и и про удовольствия, – в отржанившийся нагло, хоть со стороны уже последний ввечер и по сапожом Петра Петровича, – пробавил Васин коли почти. Говорил он из тополу укусил, что ыстановлялся в почь улыбнулся ей денег, но отчетая Алексей. Ибо мне касается останется выше на стол, чтобы они не только были извсех и одеяло приготовее, и… собой? (и вы по порпашу".

– Ну, не только удараетесь, – отвечал крикованький и рот Марья, – правню той неизвидений сюач, на света-с. Хохлактива, так пронюблю поверительность сознать. Соймет милый ей выйдет в то же время, оза что теперь самый друг полый, к азношений чувств, он отчеб будто сдуновался хотел всё долом.

- И что именно?

- Оробеть; дома, Лебядкин еще наемное, - обракалась Лизавета! - кроме в последних, вдруг сделающих грудих и кипел паннюю, все находящую (чем нашам готов было условно захлоее пожалеь, и выротили за его), по 1860 - готовую часа, «ю, когда читали в буюшу» волосов и вообще под их пом

In [16]:
loss.item()

1.419751763343811

In [25]:
with torch.no_grad():
    xt, yt = batch("val")
    logt, losst = m2(xt.to(device), yt.to(device))

In [26]:
losst

tensor(1.4779, device='cuda:0')

In [21]:
xt.shape

torch.Size([128, 128])